<a href="https://colab.research.google.com/github/YoonSungLee/Galaxies-Classification-By-Using-Deep-Learning/blob/master/Pratice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 구글 드라이브 mount

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# 패키지 불러오기
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
# 랜덤시드 고정
np.random.seed(3)

# 데이터 생성
# rescale: rescaling factor. Defaults to None.
# If None or 0, no rescaling is applied, otherwise we multiply the data by the value provided (before applying any other transformation).
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/gdrive/My Drive/Galaxies-Classification-By-Using-Deep-Learing/datasets/train_galaxy', # 파일 경로
    target_size=(50,50), # 이미지의 크기
    batch_size=16,# 배치 크기
    class_mode='categorical'# 분류 방식
)

val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_directory(
    '/content/gdrive/My Drive/Galaxies-Classification-By-Using-Deep-Learing/datasets/val_galaxy', # 파일 경로
    target_size=(50,50), # 이미지의 크기
    batch_size=16,# 배치 크기
    class_mode='categorical'# 분류 방식
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/content/gdrive/My Drive/Galaxies-Classification-By-Using-Deep-Learing/datasets/test_galaxy', # 파일 경로
    target_size=(50,50), # 이미지의 크기
    batch_size=16,# 배치 크기
    class_mode='categorical'# 분류 방식
)

Found 10249 images belonging to 3 classes.
Found 600 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


ImageDataGenerator를 이용하는 과정에서 batch_size의 결정이 하나의 이슈가 될 수 있다. 공개되어있는 많은 코드들을 보면 2의 제곱꼴의 형태로 이 값을 설정하는 것을 확인할 수 있다. 따라서 16으로 설정한다.

In [0]:
# 모델 설계

def create_model():
    n_kernels = 16
    model = Sequential()

    # N, 50, 50, 3
    model.add(Conv2D(n_kernels*2, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal', padding='valid', input_shape=(50,50,3)))
    # N, 48, 48, 32
    model.add(BatchNormalization())
    # N, 48, 48, 32
    model.add(MaxPooling2D(pool_size=(2,2)))
    # N, 24, 24, 32
    model.add(Conv2D(n_kernels*2, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal', padding='valid'))
    # N, 22, 22, 32
    model.add(Conv2D(n_kernels*4, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal', padding='valid'))
    # N, 20, 20, 64
    model.add(BatchNormalization())
    # N, 20, 20, 64
    model.add(MaxPooling2D(pool_size=(2,2)))
    # N, 10, 10, 64
    model.add(Conv2D(n_kernels*4, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal', padding='valid'))
    # N, 8, 8, 64
    model.add(Conv2D(n_kernels*8, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal', padding='valid'))
    # N, 6, 6, 128
    model.add(BatchNormalization())
    # N, 6, 6, 128
    model.add(MaxPooling2D(pool_size=(2,2)))
    # N, 3, 3, 128

    model.add(GlobalAveragePooling2D())
    # N, 128
    # model.add(Dense(128, activation='relu'))
    # model.add(Dropout(0.3))
    model.add(Dense(3, activation='softmax'))
    # N, 3

    # 모델 학습과정 설정
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

    return model

### 모델 설계
* AI Innovation Square에서 배운 CNN의 몇 가지 모델링 기법을 적용했다.
* feature map이 홀수가 되는 것을 방지하기 위해 첫 번째 Convolution Layer는 1개만 쌓는 방식을 택했다.
* 과적합을 방지하고 정규화를 위해 Convolution Layer 사이에 BatchNormalization을 적용했다.
* 기존의 CNN에서 feature map을 flatten하는 방식이 비효율적이라고 판단하여 새롭게 등장한 방법인 GlobalAveragePoolin2D를 적용하여 마지막 Layer와 연결했다(이는 각 feature map의 평균을 구해서 flatten처럼 길게 배열).
* 마지막 Layer에서 Dense Layer를 더 추가하지 않았다. 만약 추가한다면 Dense Layer 사이에 Dropout을 이용하여 과적합을 방지하는 전략을 써야 할 것이다.
* Convolution Layer : activation='relu'(CNN에서 좋은 효과를 보이는 활성화함수 적용)
* Convolution Layer : kernel_initializer='he_normal'(activation function을 relu로 설정했기 때문)
* Dense Layer : activation='softmax'(다중 분류문제이기 때문)

### Early Stopping 의 개념과 Keras 를 통한 구현
* https://3months.tistory.com/424